In [1]:
from torch_geometric.datasets import EllipticBitcoinDataset

# Load the dataset
root = 'EllipticBitcoin'
dataset = EllipticBitcoinDataset(root)
data = dataset[0]

In [2]:
data.x.shape

torch.Size([203769, 165])

In [4]:
data.x

tensor([[-0.1715, -0.1847, -1.2014,  ..., -0.0975, -0.1206, -0.1198],
        [-0.1715, -0.1847, -1.2014,  ..., -0.0975, -0.1206, -0.1198],
        [-0.1721, -0.1847, -1.2014,  ..., -0.1837, -0.1206, -0.1198],
        ...,
        [-0.1720, -0.0782,  1.0186,  ..., -0.0975, -0.1206, -0.1198],
        [-0.1728, -0.1766,  1.0186,  ..., -0.1406,  1.5197,  1.5214],
        [-0.0120, -0.1323,  0.4636,  ..., -0.1406,  1.5197,  1.5214]])

In [5]:
data.edge_index

tensor([[     0,      2,      4,  ..., 201921, 201480, 201954],
        [     1,      3,      5,  ..., 202042, 201368, 201756]])

In [4]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "k59D5ftw^N^WUm"))

cypher_query = '''
UNWIND $items AS item
MERGE (a:Transaction {id: item.id})
SET a.features = item.features, a.label = item.label
WITH a, item.edges AS edges
UNWIND edges AS edge
MERGE (b:Transaction {id: edge.dst_id})
MERGE (a)-[:PAYMENT]->(b)
'''

def create_graph(tx, batch):
    tx.run(cypher_query, items=batch)

# Define batch size
batch_size = 1000

# Upload nodes and edges in batches
with driver.session() as session:
    for i in range(0, data.num_nodes, batch_size):
        batch = []
        for node_id in range(i, min(i + batch_size, data.num_nodes)):
            node = {"id": node_id, "features": data.x[node_id].tolist(), "label": data.y[node_id].item(), "edges": []}
            node_edges = [{"src_id": node_id, "dst_id": dst.item()} for dst in data.edge_index[1, data.edge_index[0] == node_id]]
            node["edges"] = node_edges
            batch.append(node)
        session.execute_write(create_graph, batch)

driver.close()

In [ ]:
import requests

# Define batch size
batch_size = 1000

# Upload nodes and edges in batches
url = "http://localhost:5004/api/nodes/batch"
headers = {"Content-Type": "application/json"}

node_label = "Transaction"
edge_label = "PAYMENT"

for i in range(0, data.num_nodes, batch_size):
    batch = []
    for node_id in range(i, min(i + batch_size, data.num_nodes)):
        node = {"id": node_id, "features": data.x[node_id].tolist(), "label": data.y[node_id].item(), "edges": []}
        node_edges = [{"src_id": node_id, "dst_id": dst.item()} for dst in data.edge_index[1, data.edge_index[0] == node_id]]
        node["edges"] = node_edges
        batch.append(node)
    
    response = requests.post(url, json={'batch': batch, 'node_label': node_label, 'edge_label': edge_label}, headers=headers)
    if response.status_code != 200:
        print(f"Failed to upload batch starting at {i}: {response.json()}")

driver.close()


In [20]:
shortuuid.ShortUUID().random(length=16)

'bqjXN8RjXYemoSG4'

In [23]:
"1".lower()

'1'